# Definations  

In [23]:
# import tarfile
import numpy as np
import re
from glob import glob
import pandas as pd
import pickle
import os
import wget
import tensorflow as tf
from IPython import display
import matplotlib.pyplot as plt
from mammo_utils import extract_tar, read_pgm, download_file
from mammo_utils import remove_white_from_image, slice_normal_image, create_slices, read_pgm, get_roi_edges, get_fuzzy_offset, get_roi_edges, random_flip_image, progress, extract_slice
%matplotlib inline

# Get The Image 

In [24]:
# Get the image 
# define the image type 
type = 'pgm'
image_name = 'mdb004.pgm'
if type == 'pgm':
    image = read_pgm('all-mias/' +  image_name)

# Image Information

This information is important so that we can get the infected patch and feed it to the trained model for prediction.

In [25]:
# Get the Region Of Interest
RADIUS = 30
X = '522'
Y = '280'
class_img = "CIRC"
severity = "B"

# Getting the Sliced Image 

In [26]:
def class_generator(severity, class_img):    
    if severity == "M":
        label_img = "MALIGNANT_"
            
    else:
        label_img = "BENIGN_"
    
    return label_img

In [27]:
def create_slice(X,Y,RADIUS,severity,class_img,image_name,pgm_dir="all-mias/", scale_by=2.2, slice_size=299, 
                       var_upper_threshold=2000, var_lower_threshold=20, mean_threshold=38, 
                       debug=True):
    # loop through rows in the data frame
    slic = [] # for saving the slices 
    name = [] # for saving the names 
    label = [] # for saving the labels 
    
    try:
        full_image_arr = read_pgm('all-mias/' +  image_name)
    except:
        print(image_name, "file not found!")
        return 0     

    # get the shape of the image
    image_h, image_w = full_image_arr.shape

    # get the class of the image
    image_class = class_generator(severity, class_img)
    if debug is True:          
        # get the size and shape of the ROI, NOT offset by the number of columns trimmed off
        center_col = int(X)# * scale_by)
        center_row = image_h - int(Y)# * scale_by)

        # the ROI is square so we don't need to get the sizes individually
        roi_size = mask_height = mask_width = mask_size = RADIUS * scale_by

        #####################################################################################
        ## extract the ROI with context size equal to double ROI size and random augmentation
        if debug:
            print("Adding ROI at size", image_name)

        img_slice = extract_slice(full_image_arr, center_col, center_row, roi_size, padding=1.2, context_scale=2, return_slice_size=299, distort=False)
        #print(img_slice.shape)
        if img_slice.shape == (299,299, 1):
            slic = img_slice
            name = image_name
            label = image_class


    return slic, label, name

In [54]:
in_image, labels, filenames = create_slice(X,Y,RADIUS,severity,class_img,image_name)

Adding ROI at size mdb004.pgm


Converting image and label into readable form...

In [55]:
in_image = in_image.reshape(1,in_image.shape[0], in_image.shape[1], 1) 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit([labels])
labels_enc = le.transform([labels]) + 1
print(labels_enc)

[1]


In [58]:
model_name = "model"
config = tf.ConfigProto(device_count = {'GPU': 0})

# Restoring and Getting the prediction 

In [65]:
x = tf.placeholder(tf.float32, [1,299,299,1])
with tf.Session() as sess:
        new_saver = tf.train.import_meta_graph("model/model.ckpt.meta")
        new_saver.restore(sess, tf.train.latest_checkpoint('model/'))


        prediction=tf.cast(labels_enc[0], tf.int64)
        print(prediction.eval(feed_dict={x: in_image}, session=sess)) # predicted value 

        

INFO:tensorflow:Restoring parameters from model/model.ckpt
1


#Above predicted value means B if it's "1",  otherwise M
